In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython import display
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)


In [ ]:
x_train.shape,x_test.shape

In [ ]:
x_train = x_train / 255
x_test = x_test /255

In [ ]:
randInt = np.random.randint(0,x_train.shape[0]-1,30)
_,axs=plt.subplots(nrows=3,ncols=10,figsize=(24,6))
for i, ax in enumerate(axs.flat):
    ax.imshow(x_train[randInt[i]].squeeze())
plt.show()

In [ ]:
randInt = np.random.randint(0,x_test.shape[0]-1,30)
_,axs=plt.subplots(nrows=3,ncols=10,figsize=(24,6))
for i, ax in enumerate(axs.flat):
    ax.imshow(x_test[randInt[i]].squeeze())
plt.show()

In [ ]:
input_shape = (32, 32, 3)
num_classes = 10

In [ ]:
input = tf.keras.Input(shape=input_shape) 
x = tf.keras.layers.Conv2D(32,kernel_size=3, use_bias=False, padding='same')(input)
x = tf.keras.layers.BatchNormalization(center=True, scale=False)(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(32,kernel_size=3, use_bias=False, padding='same')(x)
x = tf.keras.layers.BatchNormalization(center=True, scale=False)(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(32,kernel_size=5, use_bias=False, padding='same', strides=2)(x)
x = tf.keras.layers.BatchNormalization(center=True, scale=False)(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(64,kernel_size=3, use_bias=False, padding='same')(x)
x = tf.keras.layers.BatchNormalization(center=True, scale=False)(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(64,kernel_size=3, use_bias=False, padding='same')(x)
x = tf.keras.layers.BatchNormalization(center=True, scale=False)(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(64,kernel_size=5,  use_bias=False, padding='same', strides=2)(x)
x = tf.keras.layers.BatchNormalization(center=True, scale=False)(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(128,kernel_size=4,  use_bias=False, padding='same', strides=2)(x)
x = tf.keras.layers.BatchNormalization(center=True, scale=False)(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(num_classes,activation="softmax")(x)
model = tf.keras.Model(input,output,name="convnet")
model.summary()

In [ ]:
batch_size = 32
epochs = 15

def scheduler(epoch, lr):
    if epoch < 3:
        return lr
    else:

        return lr * tf.math.exp(-0.1)

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
   

    # Check accuracy
        val_acc = 0.993
        val_loss = 0.025
        if(logs.get('val_accuracy') > val_acc) and ((logs.get('val_loss') < val_loss)):

            # Stop if threshold is met
            print(f"\n\nval_accuracy is reached {val_acc} and val_loss lower than {val_loss} so cancelling training!")
            print("")
            self.model.stop_training = True

stopCallback = myCallback()
lr = tf.keras.callbacks.LearningRateScheduler(scheduler)
tf.keras.callbacks.Callback()
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=[lr,stopCallback],validation_data=(x_test,y_test))

In [ ]:
_,ax = plt.subplots( ncols=2,nrows=1,figsize=(16,4))
ax[0].set_xlabel("epochs")
ax[0].set_ylabel("loss")
ax[0].set_title("final val_loss %1.4f"%(model.history.history["val_loss"][-1:][0]))
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("accuracy")
ax[1].set_title("final val_acc %1.4f"%(model.history.history["val_accuracy"][-1:][0]))
pd.DataFrame([model.history.history["accuracy"],model.history.history["val_accuracy"]],index=["accuracy","val_accuracy"]).T.plot(ax=ax[1])
pd.DataFrame([model.history.history["loss"],model.history.history["val_loss"]],index=["loss","val_loss"]).T.plot(ax=ax[0])


In [ ]:
true_y = np.argmax(y_test,axis=1)
true_y

In [ ]:
pred = np.argmax(model.predict(x_test),axis=1)

In [ ]:
pred

In [ ]:
randInt = np.random.randint(0,x_test.shape[0]-1,30)
_,axs=plt.subplots(nrows=5,ncols=6,figsize=(24,14))
plt.subplots_adjust(wspace=-0.6, hspace=0.6)
for i, ax in enumerate(axs.flat):
    ax.imshow(x_test4[randInt[i]].squeeze())

    if pred[randInt[i]] == true_y[randInt[i]]:
        ax.set_title( f"pred: {class_names[pred[randInt[i]]]} , \ntrue: {class_names[true_y[randInt[i]]]}" ,fontdict={"color":"green"})
    else: 
        ax.set_title( f"pred: {class_names[pred[randInt[i]]]} , \ntrue: {class_names[true_y[randInt[i]]]}" ,fontdict={"color":"red"})
    ax.set_xticks([])
    ax.set_yticks([])     
    
plt.show()

In [ ]:
def noise(array): # https://keras.io/examples/vision/autoencoder/
    """
    Adds random noise to each image in the supplied array.
    """

    noise_factor = 0.1
    noisy_array = array + noise_factor * np.random.normal(
        loc=0.0, scale=1.0, size=array.shape
    )

    return np.clip(noisy_array, 0.0, 1.0)

In [ ]:
x_train2 = noise(x_train)
x_test2 = noise(x_test)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
def image_gen(rotation_range,zoom_range,width_shift,height_shift,train,labels,end_slice): #end_slice = train.shape[0]
    datagen = ImageDataGenerator(
                                    rotation_range=rotation_range,  
                                    zoom_range = zoom_range,  
                                    width_shift_range=width_shift, 
                                    height_shift_range=height_shift)
    first = np.arange(0,int(end_slice),32)
    end = np.arange(32,int(end_slice)+1,32)
    train44=[]
    labels44=[]

    for i in range(end.shape[0]):
        train4, labels4 = datagen.flow(train[first[i]:end[i]],labels[first[i]:end[i]]).next()
        train44.append(train4)
        labels44.append(labels4[0:int(end_slice)])

    gen_train = np.concatenate(train44,axis=0)
    gen_labels = np.concatenate(labels44,axis=0)

    print("gen_train_shape:",gen_train.shape,"gen_labels_shape:",gen_labels.shape)
    train44=[]
    labels44=[]
    return gen_train, gen_labels

In [ ]:
x_train3,y_train3 = image_gen(rotation_range=10,zoom_range=0.1,width_shift=0.1,height_shift=0.1,train=x_train,labels=y_train,end_slice=x_train.shape[0]/4)
x_test3,y_test3 = image_gen(rotation_range=10,zoom_range=0.1,width_shift=0.1,height_shift=0.1,train=x_test,labels=y_test,end_slice=x_test.shape[0]/4)


In [ ]:
x_train4 = np.concatenate([x_train,x_train2,x_train3],axis=0)
y_train4 = np.concatenate([y_train,y_train,y_train3],axis=0)
x_test4 = np.concatenate([x_test,x_test2,x_test3],axis=0)
y_test4 = np.concatenate([y_test,y_test,y_test3],axis=0)

In [ ]:
print(x_train4.shape,y_train4.shape)
print(x_test4.shape,y_test4.shape)

In [ ]:
batch_size = 32
epochs = 15

stopCallback = myCallback()
lr = tf.keras.callbacks.LearningRateScheduler(scheduler)
tf.keras.callbacks.Callback()
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train4, y_train4, batch_size=batch_size, epochs=epochs, callbacks=[lr,stopCallback],validation_data=(x_test4,y_test4))

In [ ]:
_,ax = plt.subplots( ncols=2,nrows=1,figsize=(16,4))
ax[0].set_xlabel("epochs")
ax[0].set_ylabel("loss")
ax[0].set_title("final val_loss %1.4f"%(model.history.history["val_loss"][-1:][0]))
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("accuracy")
ax[1].set_title("final val_acc %1.4f"%(model.history.history["val_accuracy"][-1:][0]))
pd.DataFrame([model.history.history["accuracy"],model.history.history["val_accuracy"]],index=["accuracy","val_accuracy"]).T.plot(ax=ax[1])
pd.DataFrame([model.history.history["loss"],model.history.history["val_loss"]],index=["loss","val_loss"]).T.plot(ax=ax[0])


In [ ]:
class_names = [
    "Airplane",
    "Automobile",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck",
]

In [ ]:
x_test4.shape

In [ ]:
true_y = np.argmax(y_test4,axis=1)
true_y

In [ ]:
pred = np.argmax(model.predict(x_test4),axis=1)

In [ ]:
pred

In [ ]:
randInt = np.random.randint(0,x_test4.shape[0]-1,30)
_,axs=plt.subplots(nrows=5,ncols=6,figsize=(24,14))
plt.subplots_adjust(wspace=-0.6, hspace=0.6)
for i, ax in enumerate(axs.flat):
    ax.imshow(x_test4[randInt[i]].squeeze())

    if pred[randInt[i]] == true_y[randInt[i]]:
        ax.set_title( f"pred: {class_names[pred[randInt[i]]]} , \ntrue: {class_names[true_y[randInt[i]]]}" ,fontdict={"color":"green"})
    else: 
        ax.set_title( f"pred: {class_names[pred[randInt[i]]]} , \ntrue: {class_names[true_y[randInt[i]]]}" ,fontdict={"color":"red"})
    ax.set_xticks([])
    ax.set_yticks([])     
    
plt.show()